# Implementation and Operations Lab
<font color="red">Notice that this Lab is part C of <a href="https://github.com/ipetel/AWS_Certified_ML-acloud.guru/tree/master/CHAPTER%207%20-%20Algorithms/Algorithms%20Lab">Algorithms Lab</a></font>

<u>Use Case</u>
<p>Mr. K has given us the go to deploy the optimized Linear Learner model in production! Once this is complete Mr. K's team can use it to investigate any newly reported UFO sighting. 
Our goal is to deploy the model into production and give Mr. K's team some way to interact with the deployed model. 
</p>

<u>Goal</u>
<p>Deploy our Linear Learner model using SageMaker hosting and create a way to interact with the SageMaker endpoint created for the deployed model.</p>

<u>Notebook - Table Of Contents</u>

1. [Depoly Model](#d)
1. [Hyperparameter Tuning Job](#h)
1. [Inference using the Best Training Job](#b)
1. [Results](#res)

<u>Note</u>
1. in most of code cell that I'm using external libraries I'm importing it every time, although I can import all of them once on the start of the notebook, I wanted to show what libraries I'm using on every step.
2. there are numerous times that I'm assigning the same variables with the same values on different cells, again just want to to show what variables are used on every step.

## Depoly Model<a id='d'></a>
for deploying our trained model we have a couple of options: using SageMaker, ECS, EC2, EMR or on-premise (server or laptop or IoT device).<br>
The easiest and the least amount of effort way is using SageMaker (what a surprise!).<br><br>
In SageMaker there are two deployments types: <b>SageMaker Batch Transform</b> and <b>SageMaker Hosting Services</b>.<br> the difference between them is usage and cost.<br> 
- Batch Transform is for offline use and can generate predictions for a whole set of data all at once. 
- Hosting Services is for online use and will generate one prediction every time input will be sent to it.
<br><br>
from a cost-wise, a Batch Transform will be billed only for the prediction batch time and on finish will shut down all the used resources, In contrast to Hosting Services is keeping live endpoint on EC2 that is working 24/7.<br><br>

so now that we understand this, it looks like we will need to create a live endpoint (SageMaker Hosting Services) for a single prediction at a time.<br><br>
<u>great, so what's next? good question!</u><br>
in order to create live endpoint we will need the next steps:
1. trained model - DONE (from previous labs - <a href="https://github.com/ipetel/AWS_Certified_ML-acloud.guru/tree/master/CHAPTER%207%20-%20Algorithms/Algorithms%20Lab">Algorithms Lab</a> or <a href="https://github.com/ipetel/AWS_Certified_ML-acloud.guru/tree/master/CHAPTER%208%20-%20Evaluation%20and%20Optimization/Evaluation%20and%20Optimization%20Lab">Evaluation and Optimization Lab</a>)
1. Create a model - DONE (from previous labs - <a href="https://github.com/ipetel/AWS_Certified_ML-acloud.guru/tree/master/CHAPTER%207%20-%20Algorithms/Algorithms%20Lab">Algorithms Lab</a> or <a href="https://github.com/ipetel/AWS_Certified_ML-acloud.guru/tree/master/CHAPTER%208%20-%20Evaluation%20and%20Optimization/Evaluation%20and%20Optimization%20Lab">Evaluation and Optimization Lab</a>)
1. Create Endpoint configuration
1. Create a Endpoint

### Create Endpoint configuration + Create a Endpoint
get the best model Training Job Name (from <a href="https://github.com/ipetel/AWS_Certified_ML-acloud.guru/tree/master/CHAPTER%208%20-%20Evaluation%20and%20Optimization/Evaluation%20and%20Optimization%20Lab">Evaluation and Optimization Lab</a>)

In [1]:
import boto3

hyperparameter_tuning_job_name='ll-multiclass-HyperparamTuning-1'
tuning_job_result=boto3.client('sagemaker').describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=hyperparameter_tuning_job_name)
best_training_job_name = tuning_job_result['BestTrainingJob']['TrainingJobName']

create a model object from existing model

In [2]:
import sagemaker

# get IAM role
from sagemaker import get_execution_role
role = get_execution_role()

# Configure training job, establish SagMaker session
sess = sagemaker.Session()

# from "Evaluation and Optimization Lab" under "Inference using the Best Training Job"
model_name='best-training-job-LinearLearner-1'
bucket_name = 'allcloud-idan-aws-certified-ml-2019'

# get ECR container
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'linear-learner','1')

#for more info about "Model" go to :https://sagemaker.readthedocs.io/en/stable/model.html
best_training_job_model=sagemaker.model.Model(model_data='s3://{}/'.format(bucket_name)+'EvaluationAndOptimizationLab - 2019/output/'+best_training_job_name+'/output/model.tar.gz',
                                              role=role,
                                              image=container,
                                              name=model_name,
                                              sagemaker_session=sess)

Create Endpoint configuration + Create a Endpoint<br>
(Deploy the Model to Amazon SageMaker Hosting Services using Amazon SageMaker Python SDK)

In [3]:
#https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-deploy-model.html#ex1-deploy-model-sdk

best_training_job_model.deploy(initial_instance_count=1,
                               instance_type='ml.m4.xlarge',
                               endpoint_name='linear-learner-endpoint-1',
                               wait=True)

Using already existing model: best-training-job-LinearLearner-1


--------------------------------------------------------------------------------------------------!

create RealTimePredictor object to run prediction

In [4]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON

model_predictor = sagemaker.predictor.RealTimePredictor(endpoint='linear-learner-endpoint-1',
                                                        sagemaker_session=sess,
                                                        serializer=csv_serializer,
                                                        deserializer=json_deserializer,
                                                        content_type=CONTENT_TYPE_CSV,
                                                        accept=CONTENT_TYPE_JSON)

#for more info about 'RealTimePredictor' and 'predict' go to: https://sagemaker.readthedocs.io/en/stable/predictors.html

now that we have RealTimePredictor object we can run real time predictor<br>

In [5]:
#let's create a fake record to check the endpoint
#the order of the new_record values fit the order of the next columns: 
#[duration,latitude,longitude,researchOutcome,hasPhysicalEvidence,hadContact,shape_box,shape_circle,shape_disk,shape_light,shape_oval,shape_pyramid,shape_sphere,shape_square,shape_triangle]
new_record='50,30.30,-96.96,1,1,0,0,0,0,0,0,0,1,0'

In [15]:
prediction=model_predictor.predict(data=new_record)
predicted_label=int(prediction['predictions'][0]['predicted_label'])


researchOutcome_dict={'unexplained': 0, 'explained': 1, 'probable': 2}
predicted_researchOutcome=list(researchOutcome_dict.keys())[list(researchOutcome_dict.values()).index(predicted_label)]
print("'researchOutcome' prediction for the 'new_record' is: {}".format(predicted_researchOutcome))

'researchOutcome' prediction for the 'new_record' is: unexplained


### Notice om finish to delete the live endpoint - it's cost you money !